# Extracting historical climate (rainfall) data over selected basins

* **Products used:** 
ERA5

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

In [ ]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.load_era5 import load_era5

## Analysis Parameters

In [ ]:
# Original shapefile from https://data.apps.fao.org/map/catalog/srv/api/records/57bb1c95-2f00-4def-886f
vector_file = 'data/OB_FWR_Hydrography_Okavango_Subasins_polygon.geojson'

# define time period of interest
time_range = '2013-12', '2021-05-30'

## Define areas of interest in the Okavango Basin

In [ ]:
# load basin polygons
# Original shapefile from https://data.apps.fao.org/map/catalog/srv/api/records/57bb1c95-2f00-4def-886f-caee3d756da9
basin = gpd.read_file(vector_file)

# upstream include Cuito and Cubango subbasins
upstream = basin[basin.Subbasin.isin(['Cuito', 'Cubango'])]

# delta is part of the Okavango subbasin
delta = basin[basin.Subbasin.isin(['Okavango'])]

print(upstream)

## Retrieve historical rainfall data over all areas of interest

In [ ]:
# get historical rainfall for upstream and delta
bounds = upstream.append(delta).total_bounds
lat = bounds[1], bounds[3]
lon = bounds[0], bounds[2]

In [ ]:
# download ERA5 rainfall and aggregate to monthly
var = 'precipitation_amount_1hour_Accumulation'
precip = load_era5(var, lat, lon, time_range, reduce_func=np.sum, resample='1M').compute()

#resample to 3 month seasons
precip = precip.resample(time='3M').sum()

# fix inconsistency in axis names
precip = precip.rename({'lat':'latitude', 'lon':'longitude'})

## Create area mask for upstream and okavango regions

In [ ]:
upstream_raster = xr_rasterize(upstream, precip, x_dim='longitude', y_dim='latitude')
delta_raster = xr_rasterize(delta, precip, x_dim='longitude', y_dim='latitude')

## Calculate the total rainfall over each area

In [ ]:
upstream_rainfall = precip[var].where(upstream_raster).sum(['latitude','longitude'])
okavango_rainfall = precip[var].where(delta_raster).sum(['latitude','longitude'])

## Export results as csv

In [ ]:
upstream_rainfall.to_dataframe().drop('spatial_ref',axis=1).rename({'precipitation_amount_1hour_Accumulation':'cumulative 3-month rainfall (mm)'},axis=1).to_csv(f'results/upstream_rainfall_{time_range[0]}_to_{time_range[1]}.csv')
okavango_rainfall.to_dataframe().drop('spatial_ref',axis=1).rename({'precipitation_amount_1hour_Accumulation':'cumulative 3-month rainfall (mm)'},axis=1).to_csv(f'results/okavango_rainfall_{time_range[0]}_to_{time_range[1]}.csv')

## Visualize historical rainfall and compare to water extent changes

In [ ]:
upstream_rainfall.plot(label='Cuito + Cubango', figsize=(20,8));
okavango_rainfall.plot(label='Okavango');
plt.legend();

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

**Last Tested:**

In [ ]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')